In [2]:
#IMPORT LIBRARIES
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score

In [3]:
#LOAD DATA
df = pd.read_csv('EV_Predictive_Maintenance_Dataset_15min.csv')
df.head()

,Timestamp,SoC,SoH,Battery_Voltage,Battery_Current,Battery_Temperature,Charge_Cycles,Motor_Temperature,Motor_Vibration,Motor_Torque,...,Load_Weight,Driving_Speed,Distance_Traveled,Idle_Time,Route_Roughness,RUL,Failure_Probability,Maintenance_Type,TTF,Component_Health_Score
0,01-01-2020 00:00,0.826099,0.941338,210.163881,-22.753095,27.149201,149.190930,48.496049,0.369095,113.435589,...,741.754518,103.421162,66.232383,0.520922,0.225970,260.503381,0,1,111.116697,0.852745
1,01-01-2020 00:15,0.064728,0.916059,364.000102,-27.701120,53.655101,171.702388,57.829492,1.449195,105.587160,...,769.134035,46.041935,3.146238,0.844005,0.204350,212.813954,0,2,179.229425,0.827616
2,01-01-2020 00:30,0.873643,0.908020,388.855089,-36.646406,29.559090,191.617645,46.518363,1.859045,119.610302,...,917.262931,59.588422,79.909148,0.992405,0.175125,273.394512,0,1,171.852663,0.876887
3,01-01-2020 00:45,0.853009,0.916476,370.570602,-37.609429,29.690283,111.881817,54.163681,0.381500,182.535625,...,600.598736,44.222285,0.774000,0.007615,0.213264,229.508442,0,0,165.221328,0.816290
4,01-01-2020 01:00,0.947540,0.913206,390.011904,-14.275808,28.864338,163.774377,42.075978,0.433927,173.298044,...,613.153030,41.374684,2.872124,0.771938,0.770257,257.302631,1,0,176.890659,0.744260


In [4]:
#cheking for null values
df.isnull().sum().sum()

np.int64(0)

In [5]:
# --- BLOCK 1: SoC MONITORING (Regression) ---

df['Current_Mag'] = df['Battery_Current'].abs()
df['SoC'] = (df['Battery_Voltage'] - df['Battery_Voltage'].min()) / (df['Battery_Voltage'].max() - df['Battery_Voltage'].min())

#Feature selection
X1 = df[['Battery_Voltage', 'Battery_Current', 'Battery_Temperature']]
y1 = df['SoC']

#train test split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)
m1 = RandomForestRegressor(n_estimators=50, random_state=42).fit(X1_train, y1_train)

#accuracy score
print(f"Feature 1 (SoC) R2 Score: {r2_score(y1_test, m1.predict(X1_test)):.4f}")

# Saving (Dumping) the model
joblib.dump(m1, 'soc_model.pkl')
print("Model saved as soc_model.pkl")

Feature 1 (SoC) R2 Score: 1.0000
Model saved as soc_model.pkl


In [6]:
# --- BLOCK 2: LOW BATTERY WARNING (Classification) ---
df['Low_Battery_Alert'] = np.where(df['SoC'] < 0.15, 1, 0)

#Feature selection
X2 = df[['Battery_Voltage', 'Battery_Current', 'Battery_Temperature']]
y2 = df['Low_Battery_Alert']

#train test split
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)
m2 = RandomForestClassifier(n_estimators=50, random_state=42).fit(X2_train, y2_train)

#accuracy score
print(f"Feature 2 (Low Battery Alert) Accuracy: {accuracy_score(y2_test, m2.predict(X2_test))*100:.2f}%")

# Saving (Dumping) the model
joblib.dump(m2, 'low_battery_model.pkl')
print("Model saved as low_battery_model.pkl")

Feature 2 (Low Battery Alert) Accuracy: 100.00%
Model saved as low_battery_model.pkl


In [7]:
# --- BLOCK 3: RANGE ESTIMATION (Regression) ---
df['Remaining_Range'] = (df['SoC'] * 400) - (df['Load_Weight'] * 0.04) - (df['Route_Roughness'] * 20)

#Feature selection
X3 = df[['SoC', 'Load_Weight', 'Ambient_Temperature']]
y3 = df['Remaining_Range']

#train test split
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=42)
m3 = RandomForestRegressor(n_estimators=50, random_state=42).fit(X3_train, y3_train)

#accuracy score
print(f"Feature 3 (Range) R2 Score: {r2_score(y3_test, m3.predict(X3_test)):.4f}")

# Saving (Dumping) the model
joblib.dump(m3, 'range_model.pkl')
print("Model saved as range_model.pkl")

Feature 3 (Range) R2 Score: 0.9979
Model saved as range_model.pkl


In [8]:
# --- BLOCK 4: ABNORMAL DISCHARGE (Classification) ---
df['Abnormal_Discharge'] = np.where(((df['Current_Mag'] > 120) & (df['Driving_Speed'] < 45)) | (df['Battery_Temperature'] > 60), 1, 0)

#Feature selection
X4 = df[['Current_Mag', 'Driving_Speed', 'Motor_Torque', 'Battery_Temperature']]
y4 = df['Abnormal_Discharge']

#train test split
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.2, random_state=42)
m4 = RandomForestClassifier(n_estimators=50, random_state=42).fit(X4_train, y4_train)

#accuracy score
print(f"Feature 4 (Abnormal Discharge) Accuracy: {accuracy_score(y4_test, m4.predict(X4_test))*100:.2f}%")

# Saving (Dumping) the model
joblib.dump(m4, 'abnormal_discharge_model.pkl')
print("Model saved as abnormal_discharge_model.pkl")

Feature 4 (Abnormal Discharge) Accuracy: 100.00%
Model saved as abnormal_discharge_model.pkl


In [9]:
# --- BLOCK 5: SPEED RECOMMENDATION (Regression) ---
df['Optimal_Speed'] = 45 + (df['SoC'] * 30) - (df['Load_Weight'] * 0.01)

#Feature selection
X5 = df[['SoC', 'Load_Weight', 'Route_Roughness']]
y5 = df['Optimal_Speed']

#train test split
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.2, random_state=42)
m5 = RandomForestRegressor(n_estimators=50, random_state=42).fit(X5_train, y5_train)

#accuracy score
print(f"Feature 5 (Optimal Speed) R2 Score: {r2_score(y5_test, m5.predict(X5_test)):.4f}")

# Saving (Dumping) the model
joblib.dump(m5, 'speed_recommendation_model.pkl')
print("Model saved as speed_recommendation_model.pkl")

Feature 5 (Optimal Speed) R2 Score: 1.0000
Model saved as speed_recommendation_model.pkl


In [10]:
# --- BLOCK 6: BATTERY HEALTH (Regression) ---
df['SoH'] = (1.0 - (df['Charge_Cycles'] / 2000) - (df['Battery_Temperature'] * 0.0005)).clip(0.65, 1.0)

#Feature selection
X6 = df[['Charge_Cycles', 'Battery_Temperature', 'Component_Health_Score']]
y6 = df['SoH']

#train test split
X6_train, X6_test, y6_train, y6_test = train_test_split(X6, y6, test_size=0.2, random_state=42)
m6 = RandomForestRegressor(n_estimators=50, random_state=42).fit(X6_train, y6_train)

#accuracy score
print(f"Feature 6 (SoH) R2 Score: {r2_score(y6_test, m6.predict(X6_test)):.4f}")

# Saving (Dumping) the model
joblib.dump(m6, 'health_model.pkl')
print("Model saved as health_model.pkl")

Feature 6 (SoH) R2 Score: 1.0000
Model saved as health_model.pkl
